In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
sc = pyspark.SparkContext(appName="RDD.flatMapValues")

In [3]:
l = [('A','C'), ('A', 'D'), ('B','A'), ('C', 'A'), ('D', 'B'), ('D', 'C')]
l

[('A', 'C'), ('A', 'D'), ('B', 'A'), ('C', 'A'), ('D', 'B'), ('D', 'C')]

In [4]:
links = sc.parallelize(l).groupByKey().sortByKey().cache()
# links.collect()
links.mapValues(list).collect()

[('A', ['C', 'D']), ('B', ['A']), ('C', ['A']), ('D', ['B', 'C'])]

In [5]:
ranks = links.map(lambda item: (item[0], 1.0))
ranks.collect()

[('A', 1.0), ('B', 1.0), ('C', 1.0), ('D', 1.0)]

In [6]:
links_ranks = links.join(ranks).sortByKey()
# links_ranks.collect()
links_ranks.mapValues(lambda x: (list(x[0]), x[1])).collect()

[('A', (['C', 'D'], 1.0)),
 ('B', (['A'], 1.0)),
 ('C', (['A'], 1.0)),
 ('D', (['B', 'C'], 1.0))]

In [7]:
def f(neighbors, rank):
    for neighbor in neighbors:
        new_rank = rank / len(neighbors)
        yield (neighbor, new_rank)

In [8]:
contribs = links_ranks.flatMapValues(lambda x: f(x[0], x[1])).sortByKey()
contribs.collect()

[('A', ('C', 0.5)),
 ('A', ('D', 0.5)),
 ('B', ('A', 1.0)),
 ('C', ('A', 1.0)),
 ('D', ('B', 0.5)),
 ('D', ('C', 0.5))]

In [9]:
sc.stop()